In [ ]:
from pueday18class import *
import paho.mqtt.client as mqtt
from time import sleep

in_topic = '/lego/in/'
out_topic = '/lego/out/'

mqtt_host = 'iot.pue.es'
mqtt_port = 8883

### Define Objects
##Mqtt client object
client = mqtt.Client('rpi00')
#client.tls_set("ca.pem")
#client.tls_set("ca.pem", "rpi00.pem", "rpi00.key")

##in_topics
door_topic = in_topic+'door'
led_topic = in_topic+'led'
take_photo = in_topic+'take_photo'

##out_topics
doorbell_topic = out_topic+'doorbell'
temp_topic = out_topic+'temp'
hum_topic = out_topic+'hum'
ldr_topic = out_topic+'ldr'
photo_topic = out_topic+'photo'

##define lego house actuators
# door on BCM port 4 
lego_door = door(4)

# List of leds, now 1 led on grovepi port 7
lego_leds = leds([7,])





##Define lego house sensors


lego_temp_hum = dht_sensor(4,2,client,temp_topic, hum_topic)
lego_ldr = ldr_sensor(2,0.5,client,ldr_topic)
lego_picam = photo(client,photo_topic)
lego_doorbell = doorbell(3,2,client,doorbell_topic,lego_picam)


### MQTT on_message function 
def on_message(client, userdata, msg):

    try:
        
        if msg.topic == door_topic:
            if int(msg.payload) == 1:
                lego_door.open()
            elif int(msg.payload) == 0:
                lego_door.close()
        elif msg.topic == led_topic:
            if int(msg.payload) == 1:
                lego_leds.on()
            elif int(msg.payload) == 0:
                lego_leds.off()
        elif msg.topic == take_photo and int(msg.payload) == 1:
            lego_picam.take()
    except:
        pass



client.connect_async(mqtt_host, mqtt_port)
reconnect=True

while reconnect:
    try:
        client.reconnect()
    except ConnectionRefusedError:
        sleep(1)
    else:
        reconnect=False
        
client.subscribe(in_topic+"#")
client.on_message = on_message






#Start loops
lego_doorbell.start_loop_ring()
lego_temp_hum.start_loop()
lego_ldr.start_loop()
client.loop_start()

try:
    while True:
        sleep(5)
        
        
finally:
    lego_doorbell.stop_loop_ring()
    lego_temp_hum.stop_loop()
    lego_ldr.stop_loop()
    client.loop_stop()
    client.disconnect()